In [1]:
from osgeo import gdal
import ogr
import matplotlib.pyplot as plt
import numpy as np
import fiona
import xgboost
import pandas as pd

# Set seed
np.random.seed(100)

# Read files 
trainX = np.load('/scratch/slums/bl-slums/gt/LS-final-px-tr-2-Xa')
trainY = np.load('/scratch/slums/bl-slums/gt/LS-final-px-tr-2-Ya')
testX = np.load('/scratch/slums/bl-slums/gt/LS-final-px-te-2-Xa')
testY = np.load('/scratch/slums/bl-slums/gt/LS-final-px-te-2-Ya')

trainY = trainY.ravel()
testY = testY.ravel()
# Reshape training and test data to be suitable to sklearn methods
#print trainX.shape, trainY.shape, testX.shape, testY.shape
#trainX = trainX.reshape(trainX.shape[0], trainX.shape[2])
#valX = valX.reshape(valX.shape[0], valX.shape[2])
#testX = testX.reshape(testX.shape[0], testX.shape[2])
print trainX.shape, trainY.shape, testX.shape, testY.shape

'''
nTestSlums = np.sum(testY==0)

# print nTestSlums
# Pick all other attributes in number equal to nTestSlums randomly
class0Indices = np.where(testY==0)
class1Indices = np.where(testY==1)
class2Indices = np.where(testY==2)

print len(class1Indices[0])
indices1 = class1Indices[0][np.random.choice(len(class1Indices[0]), nTestSlums)]
indices2 = class2Indices[0][np.random.choice(len(class2Indices[0]), nTestSlums)]

class0X = testX[class0Indices[0],:]
class1X = testX[indices1,:]
class2X = testX[indices2,:]

print class0X.shape, class1X.shape, class2X.shape
class0Y = testY[class0Indices]
class1Y = testY[indices1]
class2Y = testY[indices2]

testX = np.vstack((class0X, class1X, class2X))
testY = np.hstack((class0Y, class1Y, class2Y))

print testX.shape, testY.shape
'''

(2321, 36) (2321,) (524, 36) (524,)


/afs/unity.ncsu.edu/users/k/kgadira/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


'\nnTestSlums = np.sum(testY==0)\n\n# print nTestSlums\n# Pick all other attributes in number equal to nTestSlums randomly\nclass0Indices = np.where(testY==0)\nclass1Indices = np.where(testY==1)\nclass2Indices = np.where(testY==2)\n\nprint len(class1Indices[0])\nindices1 = class1Indices[0][np.random.choice(len(class1Indices[0]), nTestSlums)]\nindices2 = class2Indices[0][np.random.choice(len(class2Indices[0]), nTestSlums)]\n\nclass0X = testX[class0Indices[0],:]\nclass1X = testX[indices1,:]\nclass2X = testX[indices2,:]\n\nprint class0X.shape, class1X.shape, class2X.shape\nclass0Y = testY[class0Indices]\nclass1Y = testY[indices1]\nclass2Y = testY[indices2]\n\ntestX = np.vstack((class0X, class1X, class2X))\ntestY = np.hstack((class0Y, class1Y, class2Y))\n\nprint testX.shape, testY.shape\n'

## XGBoost Classifier

# Run the next cell only if you want to use a subset of the 17 features generated:
8 Pansharpened (0-7)
8 Haralick (8-15)
1 NDBI (16)
1 Edge Density (17)


In [ ]:


trainX = np.nan_to_num(trainX)
testX = np.nan_to_num(testX)

# trainX = trainX[:,0:7]
# testX = testX[:,0:7]



In [2]:
print np.sum(trainY == 0)
print np.sum(trainY == 1)

1196
1125


##### Original source code: https://machinelearningmastery.com/tune-learning-rate-for-gradient-boosting-with-xgboost-in-python/

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot
model = xgboost.XGBClassifier(nthread=16 ,objective='binary:logistic')
learning_rate = [0.2, 0.3, 0.5, 0.7, 0.9]
n_estimators = [1000,5000,6000, 7000, 10000]
param_grid = dict(learning_rate=learning_rate, n_estimators = n_estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(trainX, trainY)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))
# plot
pyplot.errorbar(learning_rate, means, yerr=stds)
pyplot.title("XGBoost learning_rate vs Log Loss")
pyplot.xlabel('learning_rate')
pyplot.ylabel('Log Loss')
pyplot.show()

/usr/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Best: -0.064837 using {'n_estimators': 1000, 'learning_rate': 0.2}
-0.064837 (0.040836) with: {'n_estimators': 1000, 'learning_rate': 0.2}
-0.066358 (0.041438) with: {'n_estimators': 5000, 'learning_rate': 0.2}
-0.066358 (0.041438) with: {'n_estimators': 6000, 'learning_rate': 0.2}
-0.066358 (0.041438) with: {'n_estimators': 7000, 'learning_rate': 0.2}
-0.066358 (0.041438) with: {'n_estimators': 10000, 'learning_rate': 0.2}
-0.065081 (0.041926) with: {'n_estimators': 1000, 'learning_rate': 0.3}
-0.065608 (0.041337) with: {'n_estimators': 5000, 'learning_rate': 0.3}
-0.065608 (0.041338) with: {'n_estimators': 6000, 'learning_rate': 0.3}
-0.065608 (0.041338) with: {'n_estimators': 7000, 'learning_rate': 0.3}
-0.065608 (0.041338) with: {'n_estimators': 10000, 'learning_rate': 0.3}
-0.069636 (0.044057) with: {'n_estimators': 1000, 'learning_rate': 0.5}
-0.069566 (0.044096) with: {'n_estimators': 5000, 'learning_rate': 0.5}
-0.069566 (0.044096) with: {'n_estimators': 6000, 'learning_rate': 

RuntimeError: xdata and ydata must be the same length

In [6]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pickle
#kfold = StratifiedKFold(n_splits = 10, random_state=7)
xgb = xgboost.XGBClassifier(max_depth=500, n_estimators=1000, nthread=8 , objective='binary:logistic', learning_rate = 0.2 )
#results = cross_val_score(xgb, trainX, trainY, cv=kfold)
#print(results)
xgb.fit(trainX,trainY)
result = xgb.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Building','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'LS-xgboost-2-Cl-model.sav'
pickle.dump(xgb, open(fname, 'wb'))

Overall accuracy = 0.992366412214

Confusion Matrix 
 [[240   2]
 [  2 280]]

Classification Report 
              precision    recall  f1-score   support

   Building       0.99      0.99      0.99       242
      Other       0.99      0.99      0.99       282

avg / total       0.99      0.99      0.99       524




##### Original source code: http://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html

In [7]:
from sklearn.ensemble import RandomForestClassifier

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


#kfold = KFold(n_splits = 10, random_state=100)
rf = RandomForestClassifier(n_estimators = 500)
#results = cross_val_score(rf,trainX, trainY, cv=kfold)
#print results.mean(), results.std()
param_grid = {"max_depth": [3, None],
              "min_samples_split": [2, 3],
              "min_samples_leaf": [1, 3],
              "n_estimators": [500, 1000],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
grid_search = GridSearchCV(rf, param_grid = param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(trainX, trainY)
report(grid_result.cv_results_)



Model with rank: 1
Mean validation score: -0.050 (std: 0.017)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 500, 'criterion': 'entropy', 'min_samples_split': 2, 'max_depth': None}

Model with rank: 2
Mean validation score: -0.050 (std: 0.017)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 1000, 'criterion': 'entropy', 'min_samples_split': 2, 'max_depth': None}

Model with rank: 3
Mean validation score: -0.050 (std: 0.017)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 1000, 'criterion': 'entropy', 'min_samples_split': 3, 'max_depth': None}



In [8]:
rf = RandomForestClassifier(n_estimators = 500, bootstrap = False, min_samples_leaf = 1, min_samples_split = 2, max_depth = None)
rf.fit(trainX,trainY)
result = rf.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Building','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)
fname = 'LS-rf-2-Cl-model.sav'
pickle.dump(rf, open(fname, 'wb'))


Overall accuracy = 0.988549618321

Confusion Matrix 
 [[240   2]
 [  4 278]]

Classification Report 
              precision    recall  f1-score   support

   Building       0.98      0.99      0.99       242
      Other       0.99      0.99      0.99       282

avg / total       0.99      0.99      0.99       524




In [9]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(trainX,trainY)
result = nb.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Building','Other'])
print 'Overall accuracy = {}\n'.format(acc)
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'LS-gnb-2-Cl-model.sav'
pickle.dump(nb, open(fname, 'wb'))

Overall accuracy = 0.967557251908

Confusion Matrix 
 [[240   2]
 [ 15 267]]

Classification Report 
              precision    recall  f1-score   support

   Building       0.94      0.99      0.97       242
      Other       0.99      0.95      0.97       282

avg / total       0.97      0.97      0.97       524




In [10]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=100)
dt.fit(trainX,trainY)
result = dt.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Building','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'LS-dt-2-Cl-model.sav'
pickle.dump(dt, open(fname, 'wb'))

Overall accuracy = 0.969465648855

Confusion Matrix 
 [[236   6]
 [ 10 272]]

Classification Report 
              precision    recall  f1-score   support

   Building       0.96      0.98      0.97       242
      Other       0.98      0.96      0.97       282

avg / total       0.97      0.97      0.97       524




In [14]:
from sklearn.neighbors import KNeighborsClassifier
cv = StratifiedKFold(n_splits = 10, random_state =7 )
knn = KNeighborsClassifier()
n_neighbors = list(np.arange(3,11,1))
#print n_neighs
params_grid = dict(n_neighbors= n_neighbors)
knn_grid_search = GridSearchCV(estimator = knn, n_jobs = -1, param_grid = params_grid)
knn_grid_result = knn_grid_search.fit(trainX, trainY)
report(knn_grid_result.cv_results_)

knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(trainX, trainY)
result= knn.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Building','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'LS-knn-2-Cl-model.sav'
pickle.dump(knn, open(fname, 'wb'))


Model with rank: 1
Mean validation score: 0.917 (std: 0.099)
Parameters: {'n_neighbors': 10}

Model with rank: 2
Mean validation score: 0.917 (std: 0.097)
Parameters: {'n_neighbors': 7}

Model with rank: 3
Mean validation score: 0.916 (std: 0.100)
Parameters: {'n_neighbors': 8}

Overall accuracy = 0.965648854962

Confusion Matrix 
 [[242   0]
 [ 18 264]]

Classification Report 
              precision    recall  f1-score   support

   Building       0.93      1.00      0.96       242
      Other       1.00      0.94      0.97       282

avg / total       0.97      0.97      0.97       524




In [ ]:
from sklearn.svm import SVC
cv = StratifiedKFold(n_splits = 10, random_state =7 )
Cs = np.logspace(-6, -1, 10)
svmclf = SVC( kernel = 'poly')
svm_grid_search = GridSearchCV(estimator = svmclf, param_grid = dict(C=Cs), n_jobs = -1)
svm_grid_result = svm_grid_search.fit(trainX, trainY)
report(svm_grid_result.cv_results_)

In [ ]:
svmclf = SVC(C=0.00016681005372000591)
svmclf.fit(trainX, trainY)
result = svmclf.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Buildings','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

In [12]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier()
params_grid={
'learning_rate': ["constant", "invscaling", "adaptive"],
'hidden_layer_sizes': [(100,100,100,100), (100,100,100,100,100), (100,100,100,100,100,100,100,100)],
'alpha': [0.0001, 0.00001, 0.01],
'activation': ["logistic", "relu", "tanh"]
}

mlp_grid_search = GridSearchCV(estimator=mlp,param_grid=params_grid,n_jobs=-1,cv=kfold)
mlp_grid_result = mlp_grid_search.fit(trainX, trainY)
report(mlp_grid_result.cv_results_)


Model with rank: 1
Mean validation score: 0.981 (std: 0.014)
Parameters: {'alpha': 1e-05, 'activation': 'logistic', 'learning_rate': 'invscaling', 'hidden_layer_sizes': (100, 100, 100, 100)}

Model with rank: 2
Mean validation score: 0.980 (std: 0.009)
Parameters: {'alpha': 0.01, 'activation': 'logistic', 'learning_rate': 'constant', 'hidden_layer_sizes': (100, 100, 100, 100, 100)}

Model with rank: 3
Mean validation score: 0.980 (std: 0.011)
Parameters: {'alpha': 0.0001, 'activation': 'logistic', 'learning_rate': 'invscaling', 'hidden_layer_sizes': (100, 100, 100, 100, 100)}

Model with rank: 3
Mean validation score: 0.980 (std: 0.012)
Parameters: {'alpha': 1e-05, 'activation': 'logistic', 'learning_rate': 'invscaling', 'hidden_layer_sizes': (100, 100, 100, 100, 100)}



In [13]:
mlp = MLPClassifier(activation='logistic', learning_rate = 'invscaling',hidden_layer_sizes=(100, 100, 100, 100), alpha = 0.00001)
mlp.fit(trainX, trainY)
result = mlp.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Building','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'LS-mlp-2-Cl-model.sav'
pickle.dump(mlp, open(fname, 'wb'))

Overall accuracy = 0.988549618321

Confusion Matrix 
 [[242   0]
 [  6 276]]

Classification Report 
              precision    recall  f1-score   support

   Building       0.98      1.00      0.99       242
      Other       1.00      0.98      0.99       282

avg / total       0.99      0.99      0.99       524




In [ ]:
'''
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

gp = GaussianProcessClassifier(1.0 * RBF(1.0))
gp.fit(trainX, trainY)
result = gp.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Slum','Urban','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)
'''

In [15]:
from sklearn.ensemble import AdaBoostClassifier
adb = AdaBoostClassifier()
params_grid = dict(n_estimators=[50,100,500,1000,5000], learning_rate=[0.01, 0.007, 0.0001, 0.1, 0.0007])
adb_grid_search = GridSearchCV(estimator=adb, param_grid = params_grid, cv=kfold)
adb_search_result = adb_grid_search.fit(trainX, trainY)
report(adb_search_result.cv_results_)

Model with rank: 1
Mean validation score: 0.984 (std: 0.009)
Parameters: {'n_estimators': 500, 'learning_rate': 0.1}

Model with rank: 1
Mean validation score: 0.984 (std: 0.009)
Parameters: {'n_estimators': 1000, 'learning_rate': 0.1}

Model with rank: 3
Mean validation score: 0.983 (std: 0.009)
Parameters: {'n_estimators': 5000, 'learning_rate': 0.01}

Model with rank: 3
Mean validation score: 0.983 (std: 0.010)
Parameters: {'n_estimators': 5000, 'learning_rate': 0.1}



In [17]:
adb = AdaBoostClassifier(n_estimators=500, learning_rate=0.1)
adb.fit(trainX, trainY)
result = adb.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Building','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'LS-adaboost-2-Cl-model.sav'
pickle.dump(adb, open(fname, 'wb'))

Overall accuracy = 0.988549618321

Confusion Matrix 
 [[241   1]
 [  5 277]]

Classification Report 
              precision    recall  f1-score   support

   Building       0.98      1.00      0.99       242
      Other       1.00      0.98      0.99       282

avg / total       0.99      0.99      0.99       524




#### Implement majority voting using all the classifiers from above (with the properly tuned parameters)

In [18]:
from sklearn.ensemble import VotingClassifier

estimators = [('xgb',xgb), ('rf',rf),('nb',nb), ('dt',dt), ('knn',knn), ('mlp',mlp), ('adb',adb)]
vclf = VotingClassifier(estimators = estimators, n_jobs = -1)
vclf.fit(trainX, trainY)
result = vclf.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Building','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'LS-majority-2-Cl-model.sav'
pickle.dump(adb, open(fname, 'wb'))

Overall accuracy = 0.996183206107

Confusion Matrix 
 [[242   0]
 [  2 280]]

Classification Report 
              precision    recall  f1-score   support

   Building       0.99      1.00      1.00       242
      Other       1.00      0.99      1.00       282

avg / total       1.00      1.00      1.00       524




In [21]:
# Now predict for the entire image
from osgeo import gdal
import ogr
import matplotlib.pyplot as plt
import numpy as np
import fiona

driver = gdal.GetDriverByName('GTiff')
#rasterFileName = "/home/gadiraju/data/bl-slums/raw-img/MUL_mosaic_415.tif" #path to raster
rasterFileName1 = "/scratch/slums/bl-slums/raw-img/landsat7/PS-feb-2002-extract.tif"
dataset1 = gdal.Open(rasterFileName1)
rasterFileName2 = "/scratch/slums/bl-slums/raw-img/landsat7/ETM-feb-2002-extract-NDBI.tif"
dataset2 = gdal.Open(rasterFileName2)
rasterFileName3 = "/scratch/slums/bl-slums/raw-img/landsat7/simple-feb-2002-extract.tif"
dataset3 = gdal.Open(rasterFileName3)
rasterFileName4 = "/scratch/slums/bl-slums/raw-img/landsat7/adv-feb-2002-extract.tif"
dataset4 = gdal.Open(rasterFileName4)
rasterFileName5 = "/scratch/slums/bl-slums/raw-img/landsat7/higher-feb-2002-extract.tif"
dataset5 = gdal.Open(rasterFileName5)


bands1=[]
bands2=[]
bands3=[]
bands4=[]
bands5=[]
data_all_bands = []
cols = dataset1.RasterXSize
rows = dataset1.RasterYSize

transform = dataset1.GetGeoTransform()


img_data = np.zeros((rows, cols,36)) - 1
print img_data.shape

for i in range(6):
    band = dataset1.GetRasterBand(i+1)
    data = band.ReadAsArray(0,0,cols,rows).astype(np.float)
    img_data[:,:,i] = data
    # data_all_bands.append(band.ReadAsArray(0,0,cols,rows).astype(np.float))
band = dataset2.GetRasterBand(1)
data = band.ReadAsArray(0,0,cols,rows).astype(np.float)
img_data[:,:,6] = data
for i in range(8):#simple
    band = dataset3.GetRasterBand(i+1)
    data = band.ReadAsArray(0,0,cols,rows).astype(np.float)
    img_data[:,:,7+i] = data
for i in range(10):#adv
    band = dataset4.GetRasterBand(i+1)
    data = band.ReadAsArray(0,0,cols,rows).astype(np.float)
    img_data[:,:,15+i] = data
for i in range(11):#higher
    band = dataset5.GetRasterBand(i+1) 
    data = band.ReadAsArray(0,0,cols,rows).astype(np.float)
    img_data[:,:,25+i] = data

print img_data.shape


(3139, 3125, 36)
(3139, 3125, 36)


In [24]:
import osr

def write_image(imgrows, imgcols, imgbands, imggeotrans,edges, opath):
    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(opath, imgrows, imgcols, imgbands, gdal.GDT_Float32)
    outRaster.SetGeoTransform(imggeotrans)
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(edges)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(32643)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()



In [27]:
model = rf
result_raster = np.zeros((rows, cols)) - 1
for i in range(rows):
    curr_row = img_data[i,:,:]
    result_raster[i,:] = model.predict(curr_row)
    if i%1000 == 0 and i > 0:
        print 'Finished {} rows'.format(i)
write_image(cols, rows, 1,transform, result_raster, '/scratch/slums/bl-slums/clf-img/LS-RF-Feb-2002.tif')

Finished 1000 rows
Finished 2000 rows
Finished 3000 rows


In [55]:
print np.sum(result_raster == 0)
print np.sum(result_raster == 1)
print np.sum(result_raster> 1)

4575472
5233903
0
